In [82]:
import pygmo as pg
import numpy as np
import pandas as pd
import sys, os 
main_folder = './../'
if main_folder not in sys.path:
    sys.path.append(main_folder)
from population import Population

In [87]:
Population(dimension=4, lowerLimit=-100, upperLimit=100, 
           initialPopulation=4, method='real', opposition=False).create()

array([[-56.77342672, -13.42665826, -17.85784837, -37.75237201],
       [-52.26731749, -52.97794206, -84.82683122,  47.3170003 ],
       [-76.38657582,   2.54296338, -71.57418341, -56.59349107],
       [-43.20889633, -66.54298133, -30.01805075, -95.76721267]])

In [1]:
class EvolutionaryAlgorithm:
    def __init__(self, dim=2, pop_size=10):
        self.dim            = dim
        self.pop_size       = pop_size

        self.fitnessEvals   = 0

In [762]:
class DifferentialEvolution(EvolutionaryAlgorithm):
    def __init__(self, dim=2, func_id=1, pop_size=30, 
                 crossover = 'binomial', prob_cr = .9, pop_corpus = 'real', 
                 opposition=False, mutation='best', lambda_mutation = 1, n_diff=1, 
                F = [.8], substitute = 'random'):
        # Initialize superclass EvolutionaryAlgorithm
        super().__init__(dim=dim, pop_size=pop_size)
        self.xMin       = -100       # Search space limits
        self.xMax       =  100       #

        self.func_id    = func_id

        # Parameters
        self.param_F    = F   # Mutation parameter F. If len(F) == n_diff, each F is applied 
                            # differently to each diff mutation
        self.prob_cr = prob_cr   # Crossover probability
        self.opposition = opposition
        self.pop_corpus = pop_corpus # ['real', 'integer', 'binary']
        self.mutation = mutation # ['best', 'rand', 'mixed' ]
        self.lambda_mutation = lambda_mutation # float in (0, 1)
        self.n_diff = n_diff # [1, 2, 3, ...] (usually 1 or 2)
        self.substitute = substitute # ['random', 'edge', 'none'] what to do to outside specimen
        self.crossover = crossover
        self.generations = 0
        
        self.population = None
        self.trialPopulation = None
        self.mutatedPopulation = None
        
        # Fitness Function definition
        if self.func_id < 23:
            self.problem = pg.problem(pg.cec2014(prob_id=self.func_id, dim=self.dim))
        else:
            raise NotImplementedError("f_{:2d} not yet implemented".format(self.func_id))
            return -1

        # Initialize population DataFrame and compute initial Fitness
        self.init_states()

    def __str__(self):
        return "DE/" + self.mutation + "/" + str(self.n_diff) + "/" + self.crossover[:3]
        
    def init_states(self):
        ''' Randomly initialize states and sigma values following a uniform initialization
            between limits xMax and xMin. Assign state and fitness values to self.population.

            population is a DataFrame with one row per individual, one column per
            dimension and one extra for fitness values.

            Arguments: None

            Returns: self.population, DataFrame with dimensions (population size) by (dimension + 1).
        '''
        
        specimen = Population(dimension=self.dim, lowerLimit=self.xMin, upperLimit=self.xMax, 
                       initialPopulation=self.pop_size, method=self.pop_corpus, opposition=self.opposition).create()
        
        initPopulation = pd.DataFrame(specimen)

        self.population = self.set_state(initPopulation)
        self.generations += 1
        return self.population.copy()
    
    def get_fitness(self, specimen):
        ''' Wrapper that returns fitness value for state input specimen and increments
            number of fitness evaluations.

            Argument: specimen. State vector of length (dim).

            Returns : Fitness for given input state as evaluated by target function.
        '''
        self.fitnessEvals +=1        
        return self.problem.fitness(specimen)[0]
    
    def set_state(self, newPopulation):
        ''' Function to attribute new values to a population DataFrame.
            Guarantees correct fitness values for each state update.
            Includes specimen viability evaluation and treatment.

            Arguments:
                newPopulation   : Population DataFrame with new state values
                substitute      : Indicated what to do with non-viable specimens.
                Options are as follows:
                    'random': re-initializes non-viable vectors;
                    'none'  : substitutes non-viable vectors with None over all dimensions;
                    'edge'  : clips non-viable vectors to nearest edge of search space.

            Returns: updatedPopulation, a population DataFrame with the input values
            checked for viability and updated fitness column.
        '''
        # TODO: Split set_state and Exception Treatment in different methods
        # Exception Treatment
        # Checks if states are inside search space. If not, treat exceptions
        logicArray = np.logical_or(np.less(newPopulation, self.xMin),
                                   np.greater(newPopulation, self.xMax))

        # Case/Switch python equivalent
        # Select Exception treatment type
        choices = {
            'random':
                pd.DataFrame(np.where(logicArray, 
                                      Population(dimension=self.dim, lowerLimit=self.xMin, 
                                                 upperLimit=self.xMax, initialPopulation=self.pop_size, 
                                                 method=self.pop_corpus, opposition=self.opposition).create(),
                                      newPopulation
                                     )),
            'none':
                pd.DataFrame(np.where(logicArray, np.NaN, newPopulation)),
            'edge':
                newPopulation.clip(lower=self.xMin).clip(upper=self.xMax)
        }
        updatedPopulation = choices.get(self.substitute, KeyError("Please select a valid substitute key")).copy()

        # Compute new Fitness
        fitness = updatedPopulation.apply(self.get_fitness, axis=1).copy()
        updatedPopulation = updatedPopulation.assign(Fitness=fitness)

        # Check if there is any NaN or None value in the population
        if updatedPopulation.isna().any().any():
            # Print number of NA values per specimen
            print("\nWarning: NA values found in population\n")
            print(updatedPopulation.drop(labels="Fitness", axis=1).isna().sum(axis=1))
            input()

        return updatedPopulation.copy()

    def get_fitness(self, specimen):
        '''
            Wrapper that returns fitness value for state input specimen and increments
            number of fitness evaluations.

            Argument: specimen. State vector of length (dim).

            Returns : Fitness for given input state as evaluated by target function.
        '''
        self.fitnessEvals +=1
        return self.problem.fitness(specimen)[0]
    
    def mutate_differential(self, indexes, method='best'):
        """
            method: ['best', 'rand', 'mixed']
            n_diff: [1, 2, ...] (usually 1 or 2)
            u = lambda*best + (1-lambda)*rand + F*(x_1-x_2) + F*(x3-x4) + ...
        """
        
        best_vector = self.population.iloc[self.population.idxmin(axis=0)['Fitness'], :-1]
        base_vector = self.population.iloc[indexes[0], :-1] 
        
        if self.mutation == 'rand':
            lambda_mutation = 0
        elif self.mutation == 'best':
            lambda_mutation = 1
        else:
            lambda_mutation = self.lambda_mutation
        
        result_vector = lambda_mutation*best_vector + (1-lambda_mutation)*base_vector
        
        if len(self.param_F) != self.n_diff:
            param_F = np.repeat(self.param_F[0], self.n_diff)
        
        count = 1
        for i in np.arange(self.n_diff):
            random_vector1 = self.population.iloc[indexes[count], :-1]
            random_vector2 = self.population.iloc[indexes[count+1], :-1]
            result_vector += self.param_F[i]*(random_vector1 - random_vector2)
            count += 2

        return result_vector
    
    def mutate(self):
        def make_index_list(index):
            indexList = list(range(0, self.pop_size))
            indexList.remove(index)
            return np.random.choice(indexList, size=3, replace=False)

        # Create list of random indexes
        randomIndexes = np.array(list(map(make_index_list, list(range(0, self.pop_size)))))

        # Mutate every specimen using scheme passed to mutationScheme        
        self.mutatedPopulation = pd.DataFrame(np.apply_along_axis(self.mutate_differential, axis=1, arr=randomIndexes))

        return self.mutatedPopulation

    def perform_crossover(self):
        ''' DE binomial crossover. Compose a trial vector based on each mutated
            vector and its corresponding parent. The new vector is composed following:

                u_ij = v_ij,  if j == K or rand(0,1) <= cross_rate
                       x_ij,  else

            The trial vector will use the mutated value if probability is within
            cross_rate or its columns is randomly selected by K = randint(0, dim).

            Returns self.trialPopulation DataFrame with updated fitness column
        '''
        ## Crossover
        if self.crossover == 'exponential':
            # j: starting index 
            L = np.random.geometric(.2, 1)
            j = np.random.randint(0, len(v))            
            
            newPopulation = pd.DataFrame(                
                np.where(np.isin(np.arange(self.dim), np.arange(j, j+L+1, 1)%self.dim),                         
                self.mutatedPopulation, 
                self.population.iloc[:, :-1])
            ) 

        else: # if self.crossover == 'binomial':
            # randomArray: Roll probability for each dimension of every specimen
            # randomK:     Sample one random integer K from [0, dim] for each specimen
            # maskArray:   Mask array for logical comparisons
            randomArray = np.random.rand(self.pop_size, self.dim)
            randomK     = np.random.randint(0, self.dim, size=self.pop_size)
            maskArray   = np.arange(self.dim)

            # Reshape and tile arrays
            randomK       = np.tile(randomK[:, np.newaxis], (1, self.dim))
            maskArray     = np.tile(maskArray[np.newaxis, :], (self.pop_size, 1))

            # Substitute mutatedPopulation values if
            #   (randomArray <= cross_rate) OR (randomK == column)
            #   In other words, Keep old values only if (randomArray > cross_rate) AND (randomK != column)                                   other=self.mutatedPopulation)
            newPopulation = pd.DataFrame(np.where(np.logical_or(np.less_equal(randomArray, self.prob_cr),
                                                                 np.equal(randomK, maskArray)),
                                                  self.mutatedPopulation, self.population.iloc[:, :-1]))

        # Compute new fitness values and treat exceptions
        self.trialPopulation = self.set_state(newPopulation)
        
            
        return self.trialPopulation

    def select_survivor(self):
        '''
            DE survivor selection. A mutant vector is carried over to the next
            generation if its fitness is better or equal than its parent's.

                x_i(t+1) = u_i(t), if f(u_i(t)) <= f(x_i(t))
                           x_i(t), else

            Returns updated self.population DataFrame
        '''
        # Selection: compare Fitness of Trial and Donor vectors. Substitute for new
        # values only if fitness decreases
        self.population = self.population.where(self.population["Fitness"] < self.trialPopulation["Fitness"],
                                                other=self.trialPopulation)

        return self.population

    def generate(self):        
        self.mutate()
        self.perform_crossover()
        self.select_survivor()
        self.generations += 1
        
        return self.population
    

In [763]:
de = DifferentialEvolution(dim=2, func_id=1, pop_size=10, crossover='exponential', opposition=True, mutation='mixed', lambda_mutation=.5)
print ("#Generations: ", de.generations)
print ("Mutation: ", de.mutation, 'Lambda', de.lambda_mutation, 'CR', de.crossover)
de.population

#Generations:  1
Mutation:  mixed Lambda 0.5 CR exponential


,0,1,Fitness
0,-84.092225,-64.518148,4.482661e+09
1,-60.579600,-14.307243,4.478873e+09
2,-94.154171,46.906646,1.519526e+10
3,45.609064,44.461594,1.909496e+07
4,96.854058,-73.488589,1.015174e+10
5,84.092225,64.518148,9.445081e+08
6,60.579600,14.307243,9.427790e+08
7,94.154171,-46.906646,7.577607e+09
8,-45.609064,-44.461594,1.647512e+09
9,-96.854058,73.488589,1.876226e+10


In [766]:
str(de)

'DE/mixed/1/exp'

In [774]:
for i in np.arange(100):
    generation = de.generate()
print ("#Generations: ", de.generations)
generation

#Generations:  105


,0,1,Fitness
0,50.35579,64.92671,100.0
1,50.35579,64.92671,100.0
2,50.35579,64.92671,100.0
3,50.35579,64.92671,100.0
4,50.35579,64.92671,100.0
5,50.35579,64.92671,100.0
6,50.35579,64.92671,100.0
7,50.35579,64.92671,100.0
8,50.35579,64.92671,100.0
9,50.35579,64.92671,100.0


In [775]:
de.population['Fitness'].min()

100.0

In [189]:
np.apply_along_axis(de.mutate_differential, 1, randomIndexes)

array([[ -48.05643518,   23.21405535],
       [ 140.45084149, -177.44222156],
       [ 121.82797453, -224.16748032],
       [  94.92958294, -230.12902149],
       [   2.61553182, -126.51635708],
       [  19.08234282, -106.76440647],
       [  -4.4312641 ,  -78.3005975 ],
       [  67.77122286, -108.86241803],
       [ -67.69086767,   58.24691622],
       [  33.82603534,  -66.60819962]])